# 準備工作：讀入套件

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

# 範例 1：線性迴歸模型

## a) 準備資料集

In [ ]:
n = 50
x = np.linspace(0, 5, n)
z = 1.2*x - 0.8

接著，我們將資料點畫在二維平面上

In [ ]:
plt.figure(figsize=(8, 4))
plt.subplot(1, 2, 1)
plt.scatter(x, z)
plt.title("$f(x)=1.2x-0.8$")

y = z + 0.4*np.random.randn(n)
plt.subplot(1, 2, 2)
plt.scatter(x, y)
plt.title("$\hat{f}(x)=f(x)+\epsilon, \epsilon\sim N(0, 0.4^2)$");

順便檢查一下資料的尺寸（shape）

In [ ]:
print(x.shape)
print(y.shape)

要注意到的是，進行模型時，輸入/輸出資料的長相通常會是 $(N, D)$ 的長相，所以我們需要做一點小小的調整

In [ ]:
X = x.reshape(50, 1)

## b) 打造線性迴歸模型

首先，我們從 Scikit-learn 當中讀入線性分類器模型 ``LinearRegression``

In [ ]:
from sklearn.linear_model import LinearRegression

我們先初始化一台線性迴歸模型吧！

In [ ]:
regr = LinearRegression()

接下來，就是要讓線性迴歸模型在資料上學習

In [ ]:
regr.fit(X, y)

這樣就結束了嗎？沒錯，在實務的資料科學任務中，訓練模型通常佔不到30%的比例。

那我們接下來要做什麼呢？當然就看看我們的模型訓練得好不好囉~

## c) 評估線性迴歸模型的訓練結果

來看看訓練結果，我們使用 ``predict()`` 來對輸入資料進行預測

In [ ]:
y_pred = regr.predict(X)

接著將圖畫出來，就能看出模型的預測值與資料集的差異

In [ ]:
plt.figure(figsize=(4, 4))
plt.scatter(x, y)
plt.plot(x, y_pred, 'r');

如果我們想知道訓練出來的迴歸直線的斜率（slope）和截距（intercept），可以使用 ``regr.coef_`` 和 ``regr.intercept_`` 來取得這兩些參數

In [ ]:
regr.coef_

In [ ]:
regr.intercept_

In [ ]:
plt.figure(figsize=(6, 6))
plt.scatter(x, y)
plt.plot(x, y_pred, 'r');
plt.title(f"$y = {regr.coef_[0]:.2f}x{regr.intercept_:.2f}$");

# 範例 2：線性迴歸預測股票隔日收盤價

這裡，我們要使用雅虎的股價資料進行股價預測，我們使用每一天的收盤價來預測隔一天的收盤價。

In [ ]:
!pip install --upgrade pandas-datareader

In [ ]:
from pandas_datareader import data as pdr
import yfinance as yfin

## a) 準備資料集

In [ ]:
yfin.pdr_override()

df = pdr.get_data_yahoo('AAPL')
df = df[-300:]

In [ ]:
df.head()

In [ ]:
df.plot(y='Close', figsize=(12, 6));

In [ ]:
date = df.index
close_price = df.Close.values
X = close_price[:-1].reshape(299, 1)
Y = close_price[1:]

In [ ]:
print(X.shape)
print(Y.shape)

我們將後面 $40$ 筆資料作為測試資料，前面 $299-40=259$ 筆當作訓練資料。

In [ ]:
t_train = date[:-40]
x_train = X[:-40]
y_train = Y[:-40]

t_test = date[-40:]
x_test = X[-40:]
y_test = Y[-40:]

In [ ]:
print(x_train.shape)
print(y_train.shape)

接著，我們將資料點畫在二維平面上

In [ ]:
plt.figure(figsize=(6, 6))
plt.scatter(x_train, y_train)
plt.scatter(x_test, y_test)
plt.legend(["Train", "Test"])
plt.title("Scatter plot of $(s_t, s_{t+1})$");

不管是訓練或是測試資料，看起來都落在一條線的上下，用線性迴歸模型似乎是很合理的選擇

## b) 打造線性迴歸模型

我們先初始化一台線性迴歸模型吧！

In [ ]:
price_regr = LinearRegression()

接下來，就是要讓線性迴歸模型在資料上學習

In [ ]:
price_regr.fit(x_train, y_train)

這樣就結束了嗎？沒錯，在實務的資料科學任務中，訓練模型通常佔不到30%的比例。

那我們接下來要做什麼呢？當然就看看我們的模型訓練得好不好囉~

## c) 評估線性迴歸模型在訓練資料與測試資料上的成果



我們使用 ``predict()`` 來對輸入資料進行預測

In [ ]:
y_pred = price_regr.predict(x_train)

接著將圖畫出來，就能看出模型的預測值與資料集的差異

In [ ]:
plt.figure(figsize=(4, 4))
plt.scatter(x_train, y_train)
plt.plot(x_train, y_pred, 'r')
plt.title("Scatter plot of $(s_t, s_{t+1})$");

我們將每一天的預測價格（第2天開始)畫出來

In [ ]:
date[:-41].shape

In [ ]:
plt.figure(figsize=(8, 4))
plt.subplot(1, 2, 1)
plt.plot(t_train[1:], y_train)
plt.plot(t_train[1:], price_regr.predict(x_train))
plt.title("Prediction of Closing Price")
plt.legend(["True", "Pred."])
plt.xticks(rotation=-45)

plt.subplot(1, 2, 2)
error = y_train - price_regr.predict(x_train)
plt.plot(t_train[1:], error)
plt.hlines(0, date[1], date[-40], linestyles='dashed')
plt.title("Prediction Error")
plt.xticks(rotation=-45);

In [ ]:
plt.figure(figsize=(8, 4))
plt.subplot(1, 2, 1)
plt.plot(t_test[-40:], y_test)
plt.plot(t_test[-40:], price_regr.predict(x_test))
plt.title("Prediction of Closing Price")
plt.legend(["True", "Pred."])
plt.xticks(rotation=-45)

plt.subplot(1, 2, 2)
error = y_test - price_regr.predict(x_test)
plt.plot(t_test[-40:], error)
plt.hlines(0, date[-40], date[-2], linestyles='dashed')
plt.title("Prediction Error")
plt.xticks(rotation=-45);

## d) 連續預測

我們可以從某一天開始預測，然後用預測出的隔日收盤價，再去預測再隔一日的收盤價，以此類推，往後預測40天，看看結果是如何

In [ ]:
predicts = []
x0 = x_test[0]
for _ in range(40):
    y0 = price_regr.predict([x0])
    predicts.append(y0)
    x0 = y0

predicts = np.array(predicts).flatten()

In [ ]:
plt.figure(figsize=(4, 4))
plt.plot(t_test[-40:], y_test)
plt.plot(t_test[-40:], predicts)
plt.title("Prediction of Closing Price")
plt.legend(["True", "Pred."])
plt.xticks(rotation=-45);

# 範例 3：SVC 分類模型

我們先來看看一個簡單的練習用資料集

## a) 準備資料集

In [ ]:
x = np.array([[-3, 2], [-6, 5], [3, -4], [2, -8]])
y = np.array([1, 1, 2, 2])

接著，我們將資料點畫在二維平面上，並將點按照類別來著色

In [ ]:
plt.scatter(x[:, 0], x[:, 1], c=y, s=100);

順便檢查一下資料的尺寸（shape）

In [ ]:
print(x.shape)
print(y.shape)

## b) 用 SVC 來打造非線性分類模型

這邊要用到的是 ``SVC`` 這個基於支持向量機（SVM）的分類模型

In [ ]:
from sklearn.svm import SVC

我們先初始化一台非線性分類模型吧！

In [ ]:
clf = SVC()

接下來，就是要讓分類模型對資料進行分類學習

In [ ]:
clf.fit(x, y)

這樣就結束了嗎？沒錯，在實務的資料科學任務中，訓練模型通常佔不到30%的比例。

那我們接下來要做什麼呢？當然就看看我們的模型訓練得好不好囉~

## c) 評估 SVC 分類模型的訓練結果


來看看訓練結果，我們使用 ``predict()`` 來對輸入資料進行預測

In [ ]:
print(x)
print(y)

In [ ]:
y_pred = clf.predict(x)

接下來，我們將包含所有資料點的矩形範圍都拿來預測

In [ ]:
x1, x2 = np.meshgrid(np.arange(-7, 4, 0.02), np.arange(-9, 6, 0.02))
Z = clf.predict(np.c_[x1.ravel(), x2.ravel()])

接著將圖畫出來，就能看出模型的預測值與資料集的差異

In [ ]:
z = Z.reshape(x1.shape)
plt.contourf(x1, x2, z, alpha=0.3)
plt.scatter(x[:,0], x[:,1], s=100, c=y)

# 範例 4：在鳶尾花資料集上打造 SVC 分類器

首先，我們使用 Scikit-learn 內建的資料集

## a) 準備資料集

In [ ]:
from sklearn.datasets import load_iris

接著，我們將 IRIS 資料集讀取進來

In [ ]:
iris = load_iris()

``iris`` 是一個類似字典（dict）變數

In [ ]:
iris.keys()

我們可以透過 ``iris["DESCR"]`` 或是 ``iris.DESCR`` 來查看 IRIS 資料集的一些敘述

In [ ]:
print(iris.DESCR)

In [ ]:
X = iris.data
Y = iris.target

順便檢查一下資料的尺寸（shape）

In [ ]:
print(X.shape)
print(Y.shape)

查看一下資料 ``X``

In [ ]:
X[:10]

In [ ]:
print(iris.feature_names)

查看一下類別標籤 ``Y``

In [ ]:
print(Y)

檢查有幾個類別

In [ ]:
np.unique(Y)

In [ ]:
print(iris.target_names)

## b) 查看資料集與分割訓練/測試資料

我們將資料的任意兩個特徵拿來化散佈圖

In [ ]:
idx = [[0, 1],
       [0, 2],
       [0, 3],
       [1, 2],
       [1, 3],
       [2, 3],
       ]

In [ ]:
plt.figure(figsize=(12, 8))
for j, (p, q) in enumerate(idx):
    plt.subplot(2, 3, j+1)
    plt.scatter(X[:, p], X[:, q], c=Y)
    plt.xlabel(iris.feature_names[p])
    plt.ylabel(iris.feature_names[q])

也可以計算特徵和特徵之間的相關係數

In [ ]:
X_corr = pd.DataFrame(X, columns=iris.feature_names).corr()
plt.figure(figsize=(6, 6))
sns.heatmap(X_corr,
            cmap="coolwarm",
            vmin=-1,
            vmax=1,
            annot=True);

我們選用最高度相關的兩個特徵，也就是 ``petal length (cm)`` 和 ``petal width (cm)``

In [ ]:
X = X[:, 2:]

將資料集分成訓練與測試資料

In [ ]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(X, Y,
                                                    test_size=0.2,
                                                    random_state=87)

In [ ]:
print(x_train.shape)
print(y_train.shape)

## c) 用 SVC 來打造非線性分類模型

這邊一樣用到 ``SVC`` 這個基於支持向量機（SVM）的分類模型

我們先初始化一台非線性分類模型吧！

In [ ]:
clf = SVC()

接下來，就是要讓分類模型對資料進行分類學習

In [ ]:
clf.fit(x_train, y_train)

這樣就結束了嗎？沒錯，在實務的資料科學任務中，訓練模型通常佔不到30%的比例。

那我們接下來要做什麼呢？當然就看看我們的模型訓練得好不好囉~

## d) 評估 SVC 分類模型的訓練結果


來看看訓練結果，我們使用 ``predict()`` 來對輸入資料進行預測

In [ ]:
y_pred = clf.predict(x_test)
print(y_pred)

In [ ]:
plt.scatter(x_test[:,0], x_test[:,1], c=y_pred);

如何判斷是否預測正確呢？

In [ ]:
plt.scatter(x_test[:,0], x_test[:,1], c=y_test==y_pred);

同樣地，我們可以預測大範圍的區域內的每一個點的預測類別

In [ ]:
x1, x2 = np.meshgrid(np.arange(0, 7, 0.02), np.arange(0, 3, 0.02))
Z = clf.predict(np.c_[x1.ravel(), x2.ravel()])
Z = Z.reshape(x1.shape)

In [ ]:
plt.contourf(x1, x2, Z, cmap="coolwarm")
plt.scatter(X[:, 0], X[:, 1], c=Y);

# 範例 5：在鳶尾花資料集上打造 KNN 分類器

我們一樣在 IRIS 資料集上建立我們的分類模型，只不過這次使用的是 KNN 演算法

## a) 準備資料集

接著，我們重新將 IRIS 資料集讀取進來

In [ ]:
X = iris.data
Y = iris.target

順便檢查一下資料的尺寸（shape）

In [ ]:
print(X.shape)
print(Y.shape)

我們一樣選用最高度相關的兩個特徵，也就是 ``petal length (cm)`` 和 ``petal width (cm)``

In [ ]:
X = X[:, 2:]

將資料集分成訓練與測試資料

In [ ]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(X, Y,
                                                    test_size=0.2,
                                                    random_state=87)

In [ ]:
print(x_train.shape)
print(y_train.shape)

## c) 用 KNN 演算法 來打造分類模型


我們這邊使用的是 KNN 演算法來進行分類

In [ ]:
from sklearn.neighbors import KNeighborsClassifier as KNC

我們先初始化一台線性分類模型吧！

In [ ]:
clf = KNC(n_neighbors=5)

接下來，就是要讓分類模型對資料進行分類學習

In [ ]:
clf.fit(x_train, y_train)

這樣就結束了嗎？沒錯，在實務的資料科學任務中，訓練模型通常佔不到30%的比例。

那我們接下來要做什麼呢？當然就看看我們的模型訓練得好不好囉~

## d) 評估 KNN 分類模型的訓練結果


來看看訓練結果，我們使用 ``predict()`` 來對輸入資料進行預測

In [ ]:
y_pred = clf.predict(x_test)
print(y_pred)

In [ ]:
plt.scatter(x_test[:,0], x_test[:,1], c=y_pred);

如何判斷是否預測正確呢？

In [ ]:
plt.scatter(x_test[:,0], x_test[:,1], c=y_test==y_pred);

同樣地，我們可以預測大範圍的區域內的每一個點的類別

In [ ]:
x1, x2 = np.meshgrid(np.arange(0, 7, 0.02), np.arange(0, 3, 0.02))
Z = clf.predict(np.c_[x1.ravel(), x2.ravel()])
Z = Z.reshape(x1.shape)

In [ ]:
plt.contourf(x1, x2, Z, cmap="coolwarm")
plt.scatter(X[:, 0], X[:, 1], c=Y);

# 範例 6：進行 K-means 分群演算法

這次，我們要在簡單的資料集上嘗試進行 K-menas 分群演算法

## a) 準備資料集

我們在 2 為平面上隨便生成 100 個點

In [ ]:
X = np.random.rand(100, 2)

In [ ]:
plt.scatter(X[:,0], X[:,1], s=50);

## b) 用 K-means 演算法進行分群


我們這邊使用的是 K-means 演算法來進行分類

In [ ]:
from sklearn.cluster import KMeans

我們先初始化 K-means 演算法吧！

In [ ]:
kmeans = KMeans(n_clusters=3)

接下來，就是要讓 K-means 演算法對資料進行分群

In [ ]:
kmeans.fit(X)

這樣就結束了嗎？沒錯，在實務的資料科學任務中，訓練模型通常佔不到30%的比例。

那我們接下來要做什麼呢？當然就看看我們的模型訓練得好不好囉~

## d) 觀察 K-means 演算法的分群結果


來看看訓練結果，我們使用 ``labels_`` 來看看每個資料點分別屬於哪一群

In [ ]:
kmeans.labels_

In [ ]:
plt.scatter(X[:,0], X[:,1], c=kmeans.labels_, s=50);

同樣地，我們可以預測大範圍的區域內的每一個點的預測類別

In [ ]:
x1, x2 = np.meshgrid(np.arange(-0.2, 1.2, 0.02), np.arange(-0.2, 1.2, 0.02))
Z = kmeans.predict(np.c_[x1.ravel(), x2.ravel()])
z = Z.reshape(x1.shape)

In [ ]:
plt.contourf(x1, x2, z, alpha=0.3)
plt.scatter(X[:,0], X[:,1], s=100, c=kmeans.labels_)

# 範例 7：進行手寫辨識分類模型

這次，我們要使用手寫數字資料集 MNIST 來進行分類

In [ ]:
from sklearn.svm import LinearSVC

首先，我們使用 TensorFlow 內建的資料集

## a) 準備資料集

In [ ]:
from tensorflow.keras.datasets import mnist

接著，我們將 MNISTS 資料集讀取進來，很棒的一點是，訓練和測試資料都幫我們切好了

In [ ]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()

順便檢查一下資料的尺寸（shape）

In [ ]:
print(x_train.shape)
print(y_train.shape)

print(x_test.shape)
print(y_test.shape)

我們只取前 10% 的資料即可

In [ ]:
x_train = x_train[:6000]
y_train = y_train[:6000]
x_test = x_test[:1000]
y_test = y_test[:1000]

查看一下 ``x_train`` 中的一筆資料

In [ ]:
x_train[0]

查看一下類別標籤 ``y_train``

In [ ]:
print(y_train)

檢查有幾個類別

In [ ]:
np.unique(y_train)

In [ ]:
_, axes = plt.subplots(nrows=2, ncols=5, figsize=(10, 3))
for i, img, label in zip(range(10), x_train, y_train):
    plt.subplot(2, 5, i+1)
    plt.imshow(img, cmap="gray_r", interpolation="nearest")
    plt.title(f"Training: {label}")
    plt.axis('off');

將每一張圖片從 $28\times 28$ 拉平（flatten）成一個 $28\times28=784$ 維的向量

In [ ]:
x_train = x_train.reshape(-1, 28*28) / 255

In [ ]:
print(x_train.shape)

In [ ]:
x_test = x_test.reshape(-1, 28*28) / 255
print(x_test.shape)

## c) 用 SVC 來打造非線性分類模型

這邊一樣用到 ``SVC`` 這個基於支持向量機（SVM）的分類模型

我們先初始化一台非線性分類模型吧！

In [ ]:
clf = SVC()

接下來，就是要讓分類模型對資料進行分類學習

In [ ]:
clf.fit(x_train, y_train)

這次會稍微有點久

## d) 評估 SVC 分類模型的訓練結果


來看看訓練結果，我們使用 ``predict()`` 來對輸入資料進行預測

In [ ]:
y_pred = clf.predict(x_test)
print(y_pred)

從測試資料挑前 10 筆出來看看，並看看 SVC 模型預測的是否正確

In [ ]:
_, axes = plt.subplots(nrows=2, ncols=5, figsize=(10, 3))
for i, img, label in zip(range(10), x_test, y_pred):
    plt.subplot(2, 5, i+1)
    plt.imshow(img.reshape(28, 28), cmap="gray_r", interpolation="nearest")
    plt.title(f"Prediction: {label}")
    plt.axis('off');

資料這麼多，每筆資料又是高維度的資料，如何看出模型是否有正確從訓練資料當中學習呢？

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix

In [ ]:
print(classification_report(y_test, y_pred))

In [ ]:
sns.heatmap(confusion_matrix(y_test, y_pred),
            cmap="coolwarm",
            annot=True,
            fmt="g");